In [ ]:
!pip install biopython --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.8 MB/s eta 0:00:00


In [2]:
# https://biopython.org/docs/1.76/api/Bio.Entrez.html
from Bio import Entrez
Entrez.email = 'taniagmangolini@gmail.com'

dict_keys(['Database', 'Query', 'CorrectedQuery', 'SpelledQuery'])


## Databases

In [69]:
handle = Entrez.einfo()
record = Entrez.read(handle)
record["DbList"]

['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'structure', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'homologene', 'medgen', 'mesh', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'proteinclusters', 'pcassay', 'protfam', 'pccompound', 'pcsubstance', 'seqannot', 'snp', 'sra', 'taxonomy', 'biocollections', 'gtr']

## Espell function

In [7]:
records = Entrez.read(Entrez.espell(db='pmc', term='biobbython'))
for key, sugestions in records.items():
  print(f'{key} : {sugestions}')

Database : pmc
Query : biobbython
CorrectedQuery : biopython
SpelledQuery : ['', 'biopython']


## Esearch function
Permite fazer consultas no NCBI.

In [8]:
records = Entrez.read(Entrez.esearch(db='pmc', term='biopython'))
print(records)

{'Count': '2715', 'RetMax': '20', 'RetStart': '0', 'IdList': ['9898917', '9883684', '9886943', '9896449', '9894515', '9882952', '9877618', '9876871', '9872083', '9710682', '9837158', '9864840', '9861041', '9855806', '9850167', '9845338', '9826903', '9835423', '9837567', '9836094'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'biopython[All Fields]', 'Field': 'All Fields', 'Count': '2733', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'biopython[All Fields]'}


In [63]:
handle = Entrez.esearch(db="gene",retmax="10",term="P53 AND Homo Sapiens [organism]")
record = Entrez.read(handle)
print(record)

{'Count': '2595', 'RetMax': '10', 'RetStart': '0', 'IdList': ['7157', '128385374', '1956', '348', '7124', '7422', '3569', '7040', '4524', '2064'], 'TranslationSet': [{'From': 'Homo Sapiens[organism]', 'To': '"Homo sapiens"[Organism]'}], 'TranslationStack': [{'Term': 'P53[All Fields]', 'Field': 'All Fields', 'Count': '24210', 'Explode': 'N'}, {'Term': '"Homo sapiens"[Organism]', 'Field': 'Organism', 'Count': '307665', 'Explode': 'Y'}, 'AND'], 'QueryTranslation': 'P53[All Fields] AND "Homo sapiens"[Organism]'}


In [70]:
handle = Entrez.esearch(db="nucleotide",term="Cypripedioideae[Orgn] AND matK[Gene]")
record = Entrez.read(handle)
record["Count"]

'830'

## Esummary function
Permite buscar mais informações sobre um item.*texto em itálico*

In [14]:
records = Entrez.read(Entrez.esearch(db='pmc', term='biopython'))
papers = records['IdList']
for paper in papers[:10]:
  summary = Entrez.read(Entrez.esummary(db='pmc', id=paper))[0]
  print(summary)


{'Item': [], 'Id': '9898917', 'PubDate': '2023 Feb 3', 'EPubDate': '2023 Feb 3', 'Source': 'BMC Biol', 'AuthorList': ['Sternke-Hoffmann R', 'Pauly T', 'Norrild RK', 'Hansen J', 'Tucholski F', 'Høie MH', 'Marcatili P', 'Dupré M', 'Duchateau M', 'Rey M', 'Malosse C', 'Metzger S', 'Boquoi A', 'Platten F', 'Egelhaaf SU', 'Chamot-Rooke J', 'Fenk R', 'Nagel-Steger L', 'Haas R', 'Buell AK'], 'Title': 'Widespread amyloidogenicity potential of multiple myeloma patient-derived immunoglobulin light chains', 'Volume': '21', 'Issue': '', 'Pages': '21', 'ArticleIds': {'pubmed': [], 'medline': [], 'pmid': '0', 'doi': '10.1186/s12915-022-01506-w', 'pmcid': 'PMC9898917'}, 'DOI': '10.1186/s12915-022-01506-w', 'FullJournalName': 'BMC Biology', 'SO': '2023 Feb 3;21:21'}
{'Item': [], 'Id': '9883684', 'PubDate': '2019 Aug 22', 'EPubDate': '2019 Aug 22', 'Source': 'Bioinformatics', 'AuthorList': ['Hugoson E', 'Lam WT', 'Guy L'], 'Title': 'miComplete: weighted quality evaluation of assembled microbial genomes

In [53]:
assemblies = Entrez.read(Entrez.esearch(db='assembly', term='GCA_003713205.1'))
assembly = assemblies['IdList'][0] if assemblies['IdList'] else None
print(f'assembly: {assembly}')
if assembly:
  summary = Entrez.read(Entrez.esummary(db='assembly', id=assembly))
  summary_data = summary['DocumentSummarySet']['DocumentSummary'][0]
  print(summary_data['Taxid'], summary_data['Organism'], summary_data['SpeciesTaxid'], summary_data['SpeciesName'], summary_data['AssemblyStatus'])


assembly: 2070231
49369 Coffea eugenioides (eudicots) 49369 Coffea eugenioides Chromosome


## Efetch
Retrieves records in the requested format from a list of one or more primary IDs.
https://www.ncbi.nlm.nih.gov/books/NBK25499/table/chapter4.T._valid_values_of__retmode_and/

In [88]:
from Bio import Entrez, SeqIO

handle = Entrez.esearch(db="nucleotide",retmax="10",term="S [Gene Name] AND Sars-Cov_2 [organism]")
#handle = Entrez.esearch(db="nucleotide",retmax="10",term="S [Gene Name] AND Sars-Cov_2 [organism] AND Refseq")
record = Entrez.read(handle)
print(record)

# get first
summary = Entrez.read(Entrez.esummary(db='nucleotide', id=record["IdList"][0]))[0]
print(summary)

genbank_record = Entrez.efetch(db="nucleotide", id=record["IdList"][0],rettype="gb", retmode="text")
record = SeqIO.read(genbank_record, "genbank")
print(record)


{'Count': '3337424', 'RetMax': '10', 'RetStart': '0', 'IdList': ['2438585312', '2438585299', '2438585287', '2438585274', '2438585261', '2438585249', '2438585236', '2438585223', '2438585210', '2438585198'], 'TranslationSet': [{'From': 'Sars-Cov_2[organism]', 'To': '"Severe acute respiratory syndrome coronavirus 2"[Organism]'}], 'TranslationStack': [{'Term': 'S[Gene Name]', 'Field': 'Gene Name', 'Count': '3398891', 'Explode': 'N'}, {'Term': '"Severe acute respiratory syndrome coronavirus 2"[Organism]', 'Field': 'Organism', 'Count': '6690456', 'Explode': 'Y'}, 'AND'], 'QueryTranslation': 'S[Gene Name] AND "Severe acute respiratory syndrome coronavirus 2"[Organism]'}
{'Item': [], 'Id': '2438585312', 'Caption': 'OQ365040', 'Title': 'Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/VA-CDC-LC0997448/2023 ORF1ab polyprotein (ORF1ab), ORF1a polyprotein (ORF1ab), surface glycoprotein (S), ORF3a protein (ORF3a), envelope protein (E), membrane glycoprotein (M), ORF6 pro

In [116]:
from Bio import Entrez, SeqIO
# idtype: Specifies the type of identifier to return for sequence databases (nuccore, popset, protein). 
# By default, ESearch returns GI numbers in its output. If idtype is set to ‘acc’, ESearch will return accession.version identifiers rather than GI numbers.
handle = Entrez.esearch(db="nuccore",retmax="1", term="P53 AND Homo Sapiens [organism]", idtype='acc')
record = Entrez.read(handle)
print(record)
acc = record["IdList"][0]
print('id', acc)
handle = Entrez.efetch(db="nucleotide", id=record["IdList"][0],rettype="gb")
record = SeqIO.read(handle, "genbank")
print(record)

{'Count': '7209', 'RetMax': '1', 'RetStart': '0', 'IdList': ['NG_033079.1'], 'TranslationSet': [{'From': 'Homo Sapiens[organism]', 'To': '"Homo sapiens"[Organism]'}], 'TranslationStack': [{'Term': 'P53[All Fields]', 'Field': 'All Fields', 'Count': '56999', 'Explode': 'N'}, {'Term': '"Homo sapiens"[Organism]', 'Field': 'Organism', 'Count': '28312156', 'Explode': 'Y'}, 'AND'], 'QueryTranslation': 'P53[All Fields] AND "Homo sapiens"[Organism]'}
id NG_033079.1
ID: NG_033079.1
Name: NG_033079
Description: Homo sapiens killin, p53 regulated DNA replication inhibitor (KLLN), RefSeqGene (LRG_1087) on chromosome 10
Number of features: 12
/molecule_type=DNA
/topology=linear
/data_file_division=PRI
/date=30-JAN-2023
/accessions=['NG_033079']
/sequence_version=1
/keywords=['RefSeq', 'RefSeqGene']
/source=Homo sapiens (human)
/organism=Homo sapiens
/taxonomy=['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Mammalia', 'Eutheria', 'Euarchontoglires', 'Primates', 'Haplor

In [120]:
fasta = SeqIO.read(Entrez.efetch(db="nucleotide", id=acc, rettype="fasta"), 'fasta')
fasta

SeqRecord(seq=Seq('TTTCTACAGAAAAAAAAAAATACATTTCTTAAGCTATGTTAATATTACATTTTC...GGT'), id='NG_033079.1', name='NG_033079.1', description='NG_033079.1 Homo sapiens killin, p53 regulated DNA replication inhibitor (KLLN), RefSeqGene (LRG_1087) on chromosome 10', dbxrefs=[])

In [72]:
from Bio import Entrez, SeqIO
handle = Entrez.efetch(db="nucleotide", id="186972394",rettype="gb", retmode="text")
record = SeqIO.read(handle, "genbank")
print(record)
handle.close()

ID: EU490707.1
Name: EU490707
Description: Selenipedium aequinoctiale maturase K (matK) gene, partial cds; chloroplast
Number of features: 3
/molecule_type=DNA
/topology=linear
/data_file_division=PLN
/date=26-JUL-2016
/accessions=['EU490707']
/sequence_version=1
/keywords=['']
/source=chloroplast Selenipedium aequinoctiale
/organism=Selenipedium aequinoctiale
/taxonomy=['Eukaryota', 'Viridiplantae', 'Streptophyta', 'Embryophyta', 'Tracheophyta', 'Spermatophyta', 'Magnoliopsida', 'Liliopsida', 'Asparagales', 'Orchidaceae', 'Cypripedioideae', 'Selenipedium']
/references=[Reference(title='Phylogenetic utility of ycf1 in orchids: a plastid gene more variable than matK', ...), Reference(title='Direct Submission', ...)]
Seq('ATTTTTTACGAACCTGTGGAAATTTTTGGTTATGACAATAAATCTAGTTTAGTA...GAA')


## NCBI Global Search
Busca global em todos os databases.




In [59]:
handle = Entrez.egquery(term="biopython")
record = Entrez.read(handle)
handle.close()
for row in record["eGQueryResult"]:
    if row["Count"] != '0':
      print(row)

{'DbName': 'pubmed', 'MenuName': 'PubMed', 'Count': '41', 'Status': 'Ok'}
{'DbName': 'pmc', 'MenuName': 'PubMed Central', 'Count': '2715', 'Status': 'Ok'}
{'DbName': 'books', 'MenuName': 'Books', 'Count': '2', 'Status': 'Ok'}
{'DbName': 'pubmedhealth', 'MenuName': 'PubMed Health', 'Count': 'Error', 'Status': 'Database Error'}
{'DbName': 'ncbisearch', 'MenuName': 'Site Search', 'Count': '1', 'Status': 'Ok'}
{'DbName': 'nuccore', 'MenuName': 'Nucleotide', 'Count': '2', 'Status': 'Ok'}
{'DbName': 'sra', 'MenuName': 'SRA', 'Count': '1377', 'Status': 'Ok'}
{'DbName': 'biosystems', 'MenuName': 'BioSystems', 'Count': 'Error', 'Status': 'Database Error'}
{'DbName': 'gds', 'MenuName': 'GEO DataSets', 'Count': '17', 'Status': 'Ok'}
{'DbName': 'bioproject', 'MenuName': 'BioProject', 'Count': '1', 'Status': 'Ok'}
